<a href="https://colab.research.google.com/github/rohskopf/FitSNAP/blob/hackathon/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install LAMMPS and FitSNAP

In [2]:
!python --version

Python 3.10.11


If you are running locally and have already installed LAMMPS and FitSNAP, skip this step.

In [6]:
# Install LAMMPS with Python interface.

!apt-get update
!apt install -y cmake build-essential git ccache openmpi-bin libopenmpi-dev python3.10-venv
!pip install --upgrade pip
!pip install numpy torch scipy virtualenv psutil pandas tabulate mpi4py Cython sklearn
!pip install fitsnap3
%cd /content
!rm -rf lammps
!git clone https://github.com/lammps/lammps.git lammps
%cd /content/lammps
!rm -rf build
!mkdir build
%cd build
!cmake ../cmake -DLAMMPS_EXCEPTIONS=yes \
               -DBUILD_SHARED_LIBS=yes \
               -DMLIAP_ENABLE_PYTHON=yes \
               -DPKG_PYTHON=yes \
               -DPKG_ML-SNAP=yes \
               -DPKG_ML-IAP=yes \
               -DPKG_ML-PACE=yes \
               -DPKG_SPIN=yes \
               -DPYTHON_EXECUTABLE:FILEPATH=`which python`
!make -j 2
!make install-python

# Install FitSNAP.

%cd /content
!rm -rf FitSNAP
!git clone https://github.com/FitSNAP/FitSNAP
#!git clone -b collected-changes https://github.com/rohskopf/FitSNAP

# Set environment variables.

!$PYTHONPATH
%env PYTHONPATH=/env/python:/bin/bash:
%env LD_LIBRARY_PATH=/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/content/lammps/build

# Move into FitSNAP directory
%cd FitSNAP

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 222 kB in 1s (165 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ccache is already the newest version (3.7.7-1).
libop

# Check if Python LAMMPS is working

In [7]:
import lammps
lmp = lammps.lammps()
print(lmp)

# Basic use

Necessary imports:

In [31]:
from mpi4py import MPI
import numpy as np
from fitsnap3lib.fitsnap import FitSnap

Then set up a communicator. In this simple example we will use the world communicator, which will actually get chosen by default if you optionally choose to not specify a communicator. Important points on parallelism:

- To take advantage of MPI processes, you must put these lines in a script and run like `mpirun -np P script.py`. 
- Examples of this are shown in the `examples/library` directory.

In [32]:
# Set up your communicator.
comm = MPI.COMM_WORLD

The first mandatory step is to create input containing desired settings. These settings include everything from which descriptors to calculate, descriptor settings, which solver to use in performing a fit, how to separate the data into groups, and other options.

Import points on input settings:
- `settings` can be a dictionary defined like below, or a path to a traditional FitSNAP input script like `/path/to/Ta-example.in`.
- Some sections are not required. E.g.
    - `SCRAPER` is only required if you want to use native file scrapers.
    - `SOLVER` is only required if you want to use native model solvers and error analysis.
    - `GROUPS` is only required if you want to define groups of configurations with names and loss function weights.

In [33]:
# Create an input dictionary containing settings.

settings = \
{
"BISPECTRUM":
    {
    "numTypes": 1,
    "twojmax": 6,
    "rcutfac": 4.67637,
    "rfac0": 0.99363,
    "rmin0": 0.0,
    "wj": 1.0,
    "radelem": 0.5,
    "type": "Ta",
    "wselfallflag": 0,
    "chemflag": 0,
    "bzeroflag": 0,
    "quadraticflag": 0,
    },
"CALCULATOR":
    {
    "calculator": "LAMMPSSNAP",
    "energy": 1,
    "force": 1,
    "stress": 1
    },
"ESHIFT":
    {
    "Ta": 0.0
    },
"SOLVER":
    {
    "solver": "SVD",
    "compute_testerrs": 1,
    "detailed_errors": 1
    },
"SCRAPER":
    {
    "scraper": "JSON" 
    },
"PATH":
    {
    "dataPath": "examples/Ta_Linear_JCP2014/JSON"
    },
"OUTFILE":
    {
    "metrics": "Ta_metrics.md",
    "potential": "Ta_pot"
    },
"REFERENCE":
    {
    "units": "metal",
    "atom_style": "atomic",
    "pair_style": "hybrid/overlay zero 10.0 zbl 4.0 4.8",
    "pair_coeff1": "* * zero",
    "pair_coeff2": "* * zbl 73 73"
    },
"EXTRAS":
    {
    "dump_dataframe": 1
    },
"GROUPS":
    {
    "group_sections": "name training_size testing_size eweight fweight vweight",
    "group_types": "str float float float float float",
    "smartweights": 0,
    "random_sampling": 0,
    "Displaced_A15" :  "1.0    0.0       100             1               1.00E-08",
    "Displaced_BCC" :  "1.0    0.0       100             1               1.00E-08",
    "Displaced_FCC" :  "1.0    0.0       100             1               1.00E-08",
    "Elastic_BCC"   :  "1.0    0.0     1.00E-08        1.00E-08        0.0001",
    "Elastic_FCC"   :  "1.0    0.0     1.00E-09        1.00E-09        1.00E-09",
    "GSF_110"       :  "1.0    0.0      100             1               1.00E-08",
    "GSF_112"       :  "1.0    0.0      100             1               1.00E-08",
    "Liquid"        :  "1.0    0.0       4.67E+02        1               1.00E-08",
    "Surface"       :  "1.0    0.0       100             1               1.00E-08",
    "Volume_A15"    :  "1.0    0.0      1.00E+00        1.00E-09        1.00E-09",
    "Volume_BCC"    :  "1.0    0.0      1.00E+00        1.00E-09        1.00E-09",
    "Volume_FCC"    :  "1.0    0.0      1.00E+00        1.00E-09        1.00E-09"
    }
}

Create an instance of `fitsnap` by feeding this input dictionary, along with the optional communicator, into the `FitSnap` class.

In [34]:
fs = FitSnap(settings, comm=comm, arglist=["--overwrite"])

This creates a `fitsnap` instance which contains its own data, such as shared and distributed memory arrays in `fitsnap.pt`, and input settings in `fitsnap.config`. The shared and distributed memory arrays are associated with all processes in the supplied communicator `comm`.

Now we can use high-level library functions to perform a fit, with the following steps:
1. Scrape data to fit to. This is parallelized over all processes in `comm` with data stored in the `fitsnap.data` dictionary. Each MPI process has a different list of `data` dictionaries.
2. Calculate descriptors. This is parallelized over all processes in `comm` by operating on each configuration in `fitsnap.data`.
3. Fit potential. This is not parallelized over processes in most basic examples, e.g. the SVD solver will use all the data in a shared array on rank 0, and perform a simple least squares fit.

### 1. Scrape data

This step collects data (configurations of atoms) and injects it into a list of `FitSnap` data dictionaries. Users are 
free to do this manually using their own formats. We will explore this option later in the tutorial. Here we provide a native 
high-level function `scrape_functions()` for this purpose. As an instance owned function, `scrape_configs()` will scrape 
according to the previously input `settings`. Most high-level functions in `FitSnap` act the same way; the `settings` 
determine the state of a `FitSnap` instance which then determines the behavior of the high-level functions.

In [36]:
fs.scrape_configs()

'scrape_configs' took 477.39 ms on rank 0


This generates a list of `FitSnap` data dictionaries:

In [21]:
print(len(fs.data))

363


Each dictionary is formated like:

In [22]:
print(fs.data[0])

{'PositionsStyle': 'angstrom', 'AtomTypeStyle': 'chemicalsymbol', 'StressStyle': 'bar', 'LatticeStyle': 'angstrom', 'EnergyStyle': 'electronvolt', 'ForcesStyle': 'electronvoltperangstrom', 'File': 'A15_5.json', 'Group': 'Displaced_A15', 'Stress': array([[27656.57,   549.87,  1164.09],
       [  549.87, 24561.93, -1132.43],
       [ 1164.09, -1132.43, 25335.96]]), 'Positions': array([[1.058466e+01, 1.052281e+01, 1.054428e+01],
       [2.589060e+00, 2.625150e+00, 2.583310e+00],
       [1.334070e+00, 2.724670e+00, 3.386000e-02],
       [3.907970e+00, 2.720860e+00, 1.053991e+01],
       [7.897000e-02, 1.264300e+00, 2.656450e+00],
       [2.729000e-02, 3.964680e+00, 2.625660e+00],
       [2.718280e+00, 3.582000e-02, 1.374930e+00],
       [2.710130e+00, 1.388000e-02, 3.940290e+00],
       [5.243820e+00, 5.221000e-02, 1.059611e+01],
       [8.006760e+00, 2.682770e+00, 2.729310e+00],
       [6.670120e+00, 2.581000e+00, 5.294000e-02],
       [9.290600e+00, 2.702140e+00, 1.052770e+01],
       [5

This is the format used by `FitSnap` to feed data into LAMMPS for descriptor calculations in the next step.

### 2. Calculate descriptors

Here we use the native high-level `process_configs()` function, which does the following:
- Allocates shared memory arrays (if using MPI) to store descriptor and fitting information.
- Loop through all the configurations in the `fitsnap.data` list of dictionaries containing configuration info.
- Calculate descriptors for these configurations and store the information in the shared arrays `fitsnap.pt.shared_arrays`.

In [23]:
fs.process_configs()

'process_configs' took 3090.19 ms on rank 0


### 3. Perform fit

Fit a model with the native high-level `perform_fit()` function, which does the following:

- Solves the ML problem to get model coefficients, such as with linear regression or NNs, depending on the choice of 
  solver in the `settings` dictionary.
- Analyze errors associated with the fits, which are stored in the `fitsnap.solver.errors` dataframe.

In [24]:
fs.perform_fit()

'fit' took 58.23 ms on rank 0
'error_analysis' took 377.88 ms on rank 0


Useful objects generated by this fit:

In [25]:
# Dataframe of detailed errors per group.
print(fs.solver.errors)

                                          ncount           mae          rmse  \
Group      Weighting  Testing  Subsystem                                       
*ALL       Unweighted Training Energy        363  1.127867e-01  3.797693e-01   
                               Force       12672  7.575758e-02  1.609730e-01   
                               Stress       2178  6.833857e+04  3.817442e+05   
           weighted   Training Energy        363  2.608423e-01  6.132321e-01   
                               Force       12672  7.574500e-02  1.609730e-01   
...                                          ...           ...           ...   
Volume_FCC Unweighted Training Force         372  3.256274e-15  7.553926e-15   
                               Stress        186  3.042005e+05  1.079178e+06   
           weighted   Training Energy         31  8.120769e-01  1.181203e+00   
                               Force         372  3.256274e-24  7.553926e-24   
                               Stress   

In [ ]:
# List of fitting coefficients (for linear models).
print(fs.solver.fit)

[-2.97994849e+00 -1.14374540e-02 -7.65461855e-03 -5.02616837e-02
 -1.49917503e-01  9.46827936e-02  5.82627755e-02  6.06076097e-02
 -1.15443486e-01 -1.70155723e-01 -1.05692177e-01  3.97826631e-02
 -1.13740488e-01  4.04876497e-02 -7.26629413e-02 -6.48706053e-02
 -9.53306396e-02 -1.02394326e-01 -1.57112283e-01  4.85467075e-02
  2.49466074e-03  1.21982221e-03 -4.97372495e-02 -5.14062785e-02
 -3.41562112e-02 -1.59489125e-02 -1.50097346e-02 -6.22553797e-03
 -6.50157917e-02  3.96654127e-02  1.07549953e-02]


In [ ]:
# Dataframe containing all fitting info and metrics.
print(fs.solver.df)

         0             1             2             3             4  \
0      1.0  1.009578e+02  2.780191e+00  6.366338e-01  8.185425e+00   
1      0.0 -1.101830e+00  5.670038e-01 -4.834217e-01  4.430488e+00   
2      0.0  6.436277e+00  5.787381e-01  3.113216e-02  3.071543e+00   
3      0.0 -1.887626e+00  1.978274e-01 -6.390258e-02 -5.361083e-02   
4      0.0  6.702331e+00  3.685913e-01 -5.500523e-01  2.117131e+00   
...    ...           ...           ...           ...           ...   
15208  0.0  1.207110e+07  4.229115e+05  3.820073e+04 -2.320800e+06   
15209  0.0  1.207110e+07  4.229115e+05  3.820073e+04 -2.320800e+06   
15210  0.0 -2.498579e-10 -3.904029e-12 -2.928022e-12  1.405450e-10   
15211  0.0 -2.498579e-10  3.904029e-12  4.880036e-13  7.808058e-11   
15212  0.0 -2.498579e-10  3.904029e-12 -9.760073e-13 -7.808058e-11   

                  5             6             7             8             9  \
0     -2.940754e+00  1.049977e+00  1.263539e+00  6.486470e+01 -2.661449e+00   
1

### 4. Writing output files

In [ ]:
# Write LAMMPS potential files.
fs.output.write_lammps(fs.solver.fit)
# Write error analysis.
fs.output.write_errors(fs.solver.errors)
# Look at files:
!ls

FitSNAP.df                 examples
FitTorch_Pytorch.pt        fitsnap3
LICENSE                    fitsnap3lib
README.md                  log.lammps
Ta_Pytorch.pt              loss_vs_epochs.dat
Ta_metrics.md              pyproject.toml
Ta_pot.mod                 setup.cfg
Ta_pot.snapcoeff           tests
Ta_pot.snapparam           tools
coupling_coefficients.yace tutorial.ipynb
docs


# Perform fits on multiple instances with different settings

Let's say we want to perform multiple fits with different settings, like different `twojmax` values.

In [37]:
# Make list of twojmax values to scan:
twojmax_list = [2,4,6,8,10]
# Make list of settings for each twojmax:
from copy import deepcopy
settings_list = [deepcopy(settings) for i in twojmax_list]
for i, twojmax in enumerate(twojmax_list):
    settings_list[i]["BISPECTRUM"]["twojmax"] = twojmax

print(len(settings_list))

5


Make a list of `FitSnap` instances, each with different settings:

In [38]:
instances = [FitSnap(setting, comm=comm, arglist=["--overwrite"]) for setting in settings_list]
print(instances)

[<fitsnap3lib.fitsnap.FitSnap object at 0x7f2564232980>, <fitsnap3lib.fitsnap.FitSnap object at 0x7f2564233580>, <fitsnap3lib.fitsnap.FitSnap object at 0x7f2564233700>, <fitsnap3lib.fitsnap.FitSnap object at 0x7f256407b280>, <fitsnap3lib.fitsnap.FitSnap object at 0x7f256407bf40>]


Loop over all instances and fit:

In [40]:
for i, instance in enumerate(instances):
    print(f"--- Instance {i} with twojmax = {instance.config.sections['BISPECTRUM'].twojmax}")
    # No need to scrape configurations again, just use the previously scraped configs by injecting 
    # the previous instance data into this instance data.
    instance.process_configs(data=fs.data)
    # Perform fit using the internal fitting data of this instance.
    instance.perform_fit()
    # Grab errors.
    f_mae = instance.solver.errors['mae'][('*ALL', 'Unweighted', 'Training', 'Force')]
    e_mae = instance.solver.errors['mae'][('*ALL', 'Unweighted', 'Training', 'Energy')]

--- Instance 0 with twojmax = ['2']
'process_configs' took 3059.95 ms on rank 0
'fit' took 8.37 ms on rank 0
'error_analysis' took 325.68 ms on rank 0
--- Instance 1 with twojmax = ['4']
'process_configs' took 2074.50 ms on rank 0
'fit' took 12.44 ms on rank 0
'error_analysis' took 330.49 ms on rank 0
--- Instance 2 with twojmax = ['6']
'process_configs' took 3033.77 ms on rank 0
'fit' took 27.74 ms on rank 0
'error_analysis' took 389.21 ms on rank 0
--- Instance 3 with twojmax = ['8']
'process_configs' took 8689.68 ms on rank 0
'fit' took 55.80 ms on rank 0
'error_analysis' took 322.48 ms on rank 0
--- Instance 4 with twojmax = ['10']
'process_configs' took 15369.91 ms on rank 0
'fit' took 110.05 ms on rank 0
'error_analysis' took 565.55 ms on rank 0


Look at the errors:

In [ ]:
# Now each instance contains fitting information (configurations and their descriptors) and errors.
for instance in instances:
    # Extract specific errors from the errors dataframe.
    # NOTE: No `Testing` key will exist if no testing groups were defined in `settings`.
    ftest_mae = instance.solver.errors['mae'][('*ALL', 'Unweighted', 'Training', 'Force')]
    etest_mae = instance.solver.errors['mae'][('*ALL', 'Unweighted', 'Training', 'Energy')]
    print(f"{instance.config.sections['BISPECTRUM'].twojmax[0]} \
          {ftest_mae:0.5f}     {etest_mae:0.5f}")

2           0.39726     0.97260
4           0.15141     0.16422
6           0.07576     0.11279
8           0.06785     0.07044
10           0.05353     0.05356


#### Note on shared memory (if using MPI).
If using MPI, each instance allocates shared memory for storing the parallel arrays. Users must therefore take care to not allocate too many `FitSnap` instances, and to properly free memory associated with unused instances. We free shared array memory by overriding the `del` statement in `FitSnap`:

In [ ]:
# Free shared memory of all instances (only necessary if using MPI):
for instance in instances:
    del instance

This example looped over fits sequentially, where each fit shared the same communicator. One could however use split communicators to achieve fits in parallel.

# How to just get the descriptors for a data set?

Sometimes we want to simply extract descriptors for data analysis without going through the pain of 
performing a fit.

TODO: Show example of extracting descriptors from configs then doing data analysis (t-SNE)

### Extracting SNAP descriptors.

If we're only calculating descriptors, we just need a simple `settings` dictionary.

In [ ]:
settings = \
{
"BISPECTRUM":
    {
    "numTypes": 1,
    "twojmax": 6,
    "rcutfac": 4.67637,
    "rfac0": 0.99363,
    "rmin0": 0.0,
    "wj": 1.0,
    "radelem": 0.5,
    "type": "Ta",
    "wselfallflag": 0,
    "bzeroflag": 1,
    "bikflag": 1
    },
"CALCULATOR":
    {
    "calculator": "LAMMPSSNAP",
    "energy": 1,
    "force": 0,
    "stress": 0,
    "per_atom_energy": 1
    },
"REFERENCE":
    {
    "units": "metal",
    "atom_style": "atomic",
    "pair_style": "zero 6.0",
    "pair_coeff": "* *"
    }
}

Make an instance like usual:

In [ ]:
fs = FitSnap(settings, arglist=["--overwrite"])

Get data from ASE `Atoms` objects:

In [ ]:
from ase.io import read
frames = read("examples/Ta_XYZ/XYZ/Displaced_FCC.xyz", ":")
print(type(frames))

<class 'list'>


Use our ASE scraper to inject a list of `Atoms` objects into a particular instance:

In [ ]:
from fitsnap3lib.scrapers.ase_funcs import ase_scraper
data = ase_scraper(frames)

Loop over configurations and calculate descriptors for each separately.

In [ ]:
for i, configuration in enumerate(data):
    print(i)
    a,b,w = fs.calculator.process_single(configuration)
    print(np.shape(a))

0
(48, 30)
1
(48, 30)
2
(48, 30)
3
(48, 30)
4
(48, 30)
5
(48, 30)
6
(48, 30)
7
(48, 30)
8
(48, 30)


### Extracting ACE descriptors.

Declare settings dictionary:

In [ ]:
settings = \
{
"ACE":
    {
    "numTypes": 1,
    "ranks": "1 2 3",
    "lmax":  "1 2 2",
    "nmax": "22 2 2",
    "nmaxbase": 22,
    "rcutfac": 4.604694451,
    "lambda": 3.5,
    "type": "Ta",
    "lmin": 0,
    "bzeroflag": 1,
    "bikflag": 1,
    "RPI_heuristic": "root_SO3_span"
    },
"CALCULATOR":
    {
    "calculator": "LAMMPSPACE",
    "energy": 1,
    "force": 0,
    "stress": 0,
    "per_atom_energy": 1
    },
"REFERENCE":
    {
    "units": "metal",
    "atom_style": "atomic",
    "pair_style": "zero 6.0",
    "pair_coeff": "* *"
    }
}

Make `FitSnap` instance:

In [ ]:
fs = FitSnap(settings, arglist=["--overwrite"])

Get configurations from somewhere (e.g. ASE):

In [ ]:
from ase.io import read
from fitsnap3lib.scrapers.ase_funcs import ase_scraper
frames = read("examples/Ta_XYZ/XYZ/Displaced_FCC.xyz", ":")
data = ase_scraper(frames)

Now this `FitSnap` instance has a list of dictionaries containing structural info:

In [ ]:
print(len(data))

9


Loop over these configurations and calculate ACE descriptors:

In [ ]:
for i, configuration in enumerate(data):
    print(i)
    a,b,w = fs.calculator.process_single(configuration)
    print(np.shape(a))

0
(48, 57)
1
(48, 57)
2
(48, 57)
3
(48, 57)
4
(48, 57)
5
(48, 57)
6
(48, 57)
7
(48, 57)
8
(48, 57)


# How to process configs once then do multiple fits?

This is useful if doing many fits with the same calculator (descriptor) settings but different solver settings. We can do this by:

1. Using one `FitSnap` instance to process configs and store data in its shared arrays.
2. Using this data as input to the solver functions of another instance.

First let's make an instance for calculating descriptors.


In [ ]:
settings = \
{
"BISPECTRUM":
    {
    "numTypes": 1,
    "twojmax": 6,
    "rcutfac": 4.67637,
    "rfac0": 0.99363,
    "rmin0": 0.0,
    "wj": 1.0,
    "radelem": 0.5,
    "type": "Ta",
    "wselfallflag": 0,
    "chemflag": 0,
    "bzeroflag": 1,
    "bikflag": 1,
    "dgradflag": 1
    },
"CALCULATOR":
    {
    "calculator": "LAMMPSSNAP",
    "energy": 1,
    "force": 1,
    "per_atom_energy": 1,
    "nonlinear": 1
    },
"PYTORCH":
    {
    "layer_sizes": "num_desc 64 64 1",
    "learning_rate": 1e-4,
    "num_epochs": 10,
    "batch_size": 4, # 363 configs in entire set
    "save_state_output": "Ta_Pytorch.pt"
    },
"SOLVER":
    {
    "solver": "PYTORCH"
    },
"SCRAPER":
    {
    "scraper": "JSON" 
    },
"PATH":
    {
    "dataPath": "examples/Ta_Linear_JCP2014/JSON"
    },
"REFERENCE":
    {
    "units": "metal",
    "atom_style": "atomic",
    "pair_style": "hybrid/overlay zero 6.0 zbl 4.0 4.8",
    "pair_coeff1": "* * zero",
    "pair_coeff2": "* * zbl 73 73"
    },
"GROUPS":
    {
    "group_sections": "name training_size testing_size eweight fweight",
    "group_types": "str float float float float",
    "smartweights": 0,
    "random_sampling": 0,
    "Displaced_A15" :  "0.7 0.3 1e-2 1",
    "Displaced_BCC" :  "0.7 0.3 1e-2 1",
    "Displaced_FCC" :  "0.7 0.3 1e-2 1",
    "Elastic_BCC"   :  "0.7 0.3 1e-2 1",
    "Elastic_FCC"   :  "0.7 0.3 1e-2 1",
    "GSF_110"       :  "0.7 0.3 1e-2 1",
    "GSF_112"       :  "0.7 0.3 1e-2 1",
    "Liquid"        :  "0.7 0.3 1e-2 1",
    "Surface"       :  "0.7 0.3 1e-2 1",
    "Volume_A15"    :  "0.7 0.3 1e-2 1",
    "Volume_BCC"    :  "0.7 0.3 1e-2 1",
    "Volume_FCC"    :  "0.7 0.3 1e-2 1"
    }
}

In [ ]:
fs1 = FitSnap(settings, arglist=["--overwrite"])
fs1.scrape_configs()
fs1.process_configs()

'scrape_configs' took 286.46 ms on rank 0
'process_configs' took 3352.57 ms on rank 0


Now use the descriptor data from this instance (which is stored in `fs1.pt`), to perform many fits 
with other instances possessing different settings.

In [ ]:
# Fit with one learning rate:
settings2 = deepcopy(settings)
settings2["PYTORCH"]["learning_rate"] = 1e-3
fs2 = FitSnap(settings2, arglist=["--overwrite"])
# Fit with the shared array data from instance `fs1`.
fs2.solver.perform_fit(pt=fs1.pt)

Epoch   Train       Val     Time (s)
0  1.343e+00  2.500e-01  4.420e-01
1  1.924e-01  7.015e-02  3.336e-01
2  8.217e-02  3.406e-02  3.907e-01
3  5.060e-02  2.197e-02  3.227e-01
4  4.938e-02  2.657e-02  4.570e-01
5  3.729e-02  1.734e-02  3.321e-01
6  2.816e-02  3.344e-02  3.219e-01
7  4.257e-02  2.359e-02  3.240e-01
8  5.604e-02  4.905e-02  3.358e-01
9  2.767e-02  1.339e-02  3.334e-01


In [ ]:
# Fit with a larger learning rate:
settings3 = deepcopy(settings)
settings3["PYTORCH"]["learning_rate"] = 1e-6
fs3 = FitSnap(settings3, arglist=["--overwrite"])
# Fit with the shared array data from instance `fs1`.
fs3.solver.perform_fit(pt=fs1.pt)

Epoch   Train       Val     Time (s)
0  4.425e+00  4.091e+00  3.584e-01
1  4.552e+00  3.972e+00  3.366e-01
2  4.512e+00  4.109e+00  4.276e-01
3  4.548e+00  3.932e+00  3.567e-01
4  4.352e+00  4.061e+00  3.627e-01
5  4.393e+00  3.840e+00  3.381e-01
6  4.334e+00  3.981e+00  3.192e-01
7  4.467e+00  3.945e+00  3.190e-01
8  4.388e+00  3.946e+00  3.209e-01
9  4.420e+00  4.059e+00  3.104e-01


Get errors of the two instances.

In [ ]:
# For NNs, `solver.errors` is currently a tuple of dictionaries.
# Errors for larger learning rate:
fs2.solver.error_analysis()
(mae_f, mae_e, rmse_f, rmse_e, count_train, count_test) = fs2.solver.errors
# Look at force MAE of specific group:
print(mae_f["Displaced_A15"])

{'train': 0.11701493425093094, 'test': 0.1115132993092906}


In [ ]:
# Errors for smaller learning rate:
fs3.solver.error_analysis()
(mae_f, mae_e, rmse_f, rmse_e, count_train, count_test) = fs3.solver.errors
# Look at force MAE of specific group:
print(mae_f["Displaced_A15"])

{'train': 0.448120823658368, 'test': 0.4525303700428897}


# Hiearchical parallelism with custom communicators

These simple examples all used a single world communicator. Our design, however, allows one to create many instances each with a different communicator, to get creative with how fits are done in parallel. For example one could split the communicator among a group of processes, and then perform multiple fits *in parallel*, where each fit is performed in parallel using the processes in its communicator. This is beyond the scope of a iPython notebook since it requires making custom Python scripts with MPI.

# How do the shared arrays work?

Each `FitSnap` instance contains shared arrays inside the `snap.pt.shared_arrays` dictionary. The descriptor array is stored in `snap.pt.shared_arrays['a'].array`. The contents of this array are shared in memory between all processes in the instance communicator `snap.pt._comm` (this is the same communicator we passed when creating the instance). This means that when an element of the shared array is changed on one process in `comm`, it will change the shared array with all other processes in the same communicator. This is important because although each `snap.pt` instance is different for all processes in a communicator, the contents `snap.pt.shared_arrays['a'].array` are shared.